# Connect to MongoDB server

In [ ]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

#Setup Mongo Connection
client = MongoClient('localhost', 27017, serverSelectionTimeoutMS=5000)

#Test MongoDB connection
try:
    client.admin.command("ismaster")
except ConnectionFailure as e:
    print e.message
    exit(1)

# Extract documents from JSON file and store in collection

In [ ]:
from bson.son import SON
import json
from pymongo.collection import CodecOptions

#Open JSON file
json_file = open('data/seinfeld_eps.json')

#Read contents of json file and write to DB
json_data = json.load(json_file)
episodes = json_data['_embedded']['episodes']
database = client.get_database('tv_data')
options = CodecOptions(document_class=SON)
seinfeld_collection = database.get_collection('seinfeld_eps').with_options(codec_options=options)
seinfeld_collection.insert_many(episodes)

# Querying documents from a collection

## Retrieve stored documents with one criterion

In [ ]:
from print_utilities import print_result

#Select all episodes from Season 1
season_1_eps = seinfeld_collection.find({'season':1})
print_result(season_1_eps)

## Retrieve a document with multiple criteria

In [ ]:
#Get season 1, episode 1
season_1_ep_1 = seinfeld_collection.find_one({'season':1,'number':1})
print_result(season_1_ep_1)

## Retrieve documents using comparison operators

In [ ]:
#Get season 1 episodes after the first
season_1_gt_1 = seinfeld_collection.find({'season':1,'number':{'$gt':1}})
print_result(season_1_gt_1)

##  Retrieve documents with a regex criterion

In [ ]:
import re

#Get all episodes starting with the letter G (or 'The G')
regex = re.compile('^(The )?G.+$')
g_eps = seinfeld_collection.find({'name':regex})
print_result(g_eps)

# Creating, updating, and deleting documents from a collection

## Insert a document

In [ ]:
#Add a new episode
seinfeld_collection.insert_one({'name':'The Data Wrangler',
                                'airdate':'2017-2-17',
                                'season': 10,
                                'number': 1})

## Update existing document

In [ ]:
#Update episode
seinfeld_collection.update_one({'name':'The Data Wrangler'},
                               {'$set':{'airdate':'2017-2-24'}})

## Delete document

In [ ]:
#Delete episode
seinfeld_collection.delete_one({'name':'The Data Wrangler'})

# Aggregate Queries

## Count all documents in a collection

In [ ]:
#Count all seinfeld episodes
episode_counts = seinfeld_collection.count()
print_result(episode_counts)

## Count documents by group

In [ ]:
#Count seinfeld episodes by season
season_cts = seinfeld_collection.aggregate([{'$group':{'_id':'$season',
                                                       '_ep count':{'$sum': 1}}},
                                            {'$project':{'season':'$_id',
                                                         '_id':0,
                                                         'ep count':'$_ep count'}}])
print_result(season_cts)

## Add filter to aggregate query

In [ ]:
#Count seinfeld episodes in season 3
s3_cts = seinfeld_collection.aggregate([{'$match':{'season':3}},
                                        {'$group':{'_id':'$season',
                                                   '_ep count':{'$sum': 1}}},
                                        {'$project':{'season':'$_id',
                                                     '_id': 0,
                                                     'ep count':'$_ep count'}}])
print_result(s3_cts)

# Disconnect From MongoDB

In [ ]:
client.close()